# Wet bulb temps by county: 2000-2020

#### Import Python tools

In [1]:
%load_ext lab_black

In [47]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import us

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
today = pd.to_datetime("today").strftime("%Y-%m-%d")

---

## Geography

#### Import counties GeoJSON with selected columns

In [41]:
counties_geo = gpd.read_file(
    "/Users/stiles/data/gis/usa_counties_esri_simple_mainland.geojson"
)[["name", "state_name", "state_fips", "fips", "population", "geometry"]]

In [93]:
counties_dict = counties_geo.set_index("fips")["name"].to_dict()

In [99]:
counties_geo.plot()

---

## Read [the data](https://figshare.com/articles/dataset/Daily_County-Level_Wet-Bulb_Globe_Temperature_Universal_Thermal_Climate_Index_and_Other_Heat_Metrics_for_the_Contiguous_United_States_2000-2020/19419836)

In [53]:
src = pd.read_csv(
    "/Users/stiles/data/climate/heat-bulb-counties/data.csv", dtype={"StCoFIPS": str}
)

#### Clean up columns

In [54]:
src.drop(["Unnamed: 0"], axis=1, inplace=True)

In [55]:
src.columns = src.columns.str.lower()

#### Date formatting

In [56]:
src["date"] = pd.to_datetime(src["date"])
src["month"] = pd.to_datetime(src["date"]).dt.month
src["day"] = pd.to_datetime(src["date"]).dt.day
src["year"] = pd.to_datetime(src["date"]).dt.year

In [57]:
src.rename(
    columns={
        "stcofips": "fips",
        "tmean_c": "temp_mean",
        "netmean_c": "net_mean",
        "wbgtmean_c": "wbgt_mean",
    },
    inplace=True,
)

#### Just the columns we need

In [59]:
df = src[
    [
        "fips",
        "date",
        "month",
        "day",
        "year",
        "temp_mean",
        "net_mean",
        "wbgt_mean",
    ]
].copy()

In [60]:
df.head()

In [64]:
df["state_fips"] = df["fips"].str[:2]

#### AP states

In [61]:
fips_to_ap = us.states.mapping("fips", "ap_abbr")

In [65]:
df["state_ap"] = df["state_fips"].map(fips_to_ap)

#### County names from geo file

In [106]:
df["county_name"] = df["fips"].map(counties_dict)

In [107]:
df.head()

,fips,date,month,day,year,temp_mean,net_mean,wbgt_mean,state_fips,state_ap,county_name
0,01001,2000-01-02,1,2,2000,17.27,10.78,16.71,01,Ala.,Autauga County
1,01003,2000-01-02,1,2,2000,18.74,11.65,18.75,01,Ala.,Baldwin County
2,01005,2000-01-02,1,2,2000,16.21,10.19,15.85,01,Ala.,Barbour County
3,01007,2000-01-02,1,2,2000,16.58,10.03,16.11,01,Ala.,Bibb County
4,01009,2000-01-02,1,2,2000,15.32,8.23,15.12,01,Ala.,Blount County


---

## Aggregate

#### Months

In [111]:
month_avg = (
    df.groupby(["fips", "county_name", "state_ap", "month"])
    .agg({"temp_mean": "mean", "net_mean": "mean", "wbgt_mean": "mean"})
    .reset_index()
    .round(2)
)

In [112]:
month_avg.head()

,fips,county_name,state_ap,month,temp_mean,net_mean,wbgt_mean
0,01001,Autauga County,Ala.,1,8.15,0.75,7.32
1,01001,Autauga County,Ala.,2,10.26,3.02,9.38
2,01001,Autauga County,Ala.,3,14.17,7.44,13.11
3,01001,Autauga County,Ala.,4,18.19,11.99,16.79
4,01001,Autauga County,Ala.,5,22.77,17.37,21.09


---

## Exports

#### Slim dataframe

In [38]:
df.to_csv("/Users/stiles/data/climate/heat-bulb-counties/data_slim.csv", index=False)

#### Months

In [36]:
month_avg.to_csv(
    "/Users/stiles/data/climate/heat-bulb-counties/month_means.csv", index=False
)